In [2]:
import platform
platform.platform()

'macOS-13.0.1-arm64-arm-64bit'

In [3]:
import torch
torch.__version__

'1.13.0'

In [4]:
import transformers

In [5]:
torch.has_mps

True

In [32]:
from datasets import load_dataset

trec=load_dataset('trec', split='train[:64]')
trec

Found cached dataset trec (/Users/tanya.banerjee/.cache/huggingface/datasets/trec/default/2.0.0/f2469cab1b5fceec7249fda55360dfdbd92a7a5b545e91ea0f78ad108ffac1c2)


Dataset({
    features: ['text', 'coarse_label', 'fine_label'],
    num_rows: 64
})

In [33]:
trec[0]

{'text': 'How did serfdom develop in and then leave Russia ?',
 'coarse_label': 2,
 'fine_label': 26}

In [34]:
from transformers import AutoModel, AutoTokenizer

In [10]:
tokenizer=AutoTokenizer.from_pretrained('bert-base-uncased') #get tokenizer
model=AutoModel.from_pretrained('bert-base-uncased') #get model

Downloading:   0%|          | 0.00/28.0 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/570 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/232k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/466k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/440M [00:00<?, ?B/s]

Some weights of the model checkpoint at bert-base-uncased were not used when initializing BertModel: ['cls.seq_relationship.bias', 'cls.predictions.decoder.weight', 'cls.predictions.transform.LayerNorm.weight', 'cls.predictions.transform.dense.bias', 'cls.predictions.bias', 'cls.predictions.transform.dense.weight', 'cls.predictions.transform.LayerNorm.bias', 'cls.seq_relationship.weight']
- This IS expected if you are initializing BertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


In [36]:
tokens=tokenizer(trec['text'], max_length=512, truncation=True, padding=True) #tokenize

In [37]:
device=torch.device('mps') #the variable device means the Apple M1 GPU now
device

device(type='mps')

In [38]:
tokens #seeing what the tokenizer did

{'input_ids': [[101, 2129, 2106, 14262, 2546, 9527, 4503, 1999, 1998, 2059, 2681, 3607, 1029, 102, 0, 0, 0, 0, 0, 0, 0], [101, 2054, 3152, 2956, 1996, 2839, 4831, 6672, 11294, 1029, 102, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0], [101, 2129, 2064, 1045, 2424, 1037, 2862, 1997, 12330, 1005, 2613, 3415, 1029, 102, 0, 0, 0, 0, 0, 0, 0], [101, 2054, 1042, 5004, 2140, 13273, 1996, 17763, 2044, 1996, 2822, 2095, 1997, 1996, 10608, 1029, 102, 0, 0, 0, 0], [101, 2054, 2003, 1996, 2440, 2433, 1997, 1012, 4012, 1029, 102, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0], [101, 2054, 17152, 7028, 8040, 28819, 16570, 10312, 1996, 8513, 2013, 2026, 6265, 1029, 102, 0, 0, 0, 0, 0, 0], [101, 2054, 2136, 2106, 3598, 1005, 1055, 2358, 1012, 3434, 13240, 2468, 1029, 102, 0, 0, 0, 0, 0, 0, 0], [101, 2054, 2003, 1996, 4587, 9518, 1029, 102, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0], [101, 2054, 2024, 11290, 16285, 1029, 102, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0], [101, 2171, 1996, 11228, 1011, 4320, 17284, 4477, 1997, 1996, 2214, 2225

In [39]:
model.to(device)
# sending model to GPU

BertForSequenceClassification(
  (bert): BertModel(
    (embeddings): BertEmbeddings(
      (word_embeddings): Embedding(30522, 768, padding_idx=0)
      (position_embeddings): Embedding(512, 768)
      (token_type_embeddings): Embedding(2, 768)
      (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (encoder): BertEncoder(
      (layer): ModuleList(
        (0): BertLayer(
          (attention): BertAttention(
            (self): BertSelfAttention(
              (query): Linear(in_features=768, out_features=768, bias=True)
              (key): Linear(in_features=768, out_features=768, bias=True)
              (value): Linear(in_features=768, out_features=768, bias=True)
              (dropout): Dropout(p=0.1, inplace=False)
            )
            (output): BertSelfOutput(
              (dense): Linear(in_features=768, out_features=768, bias=True)
              (LayerNorm): LayerNorm((768,), eps=1e-12, element

In [40]:
from transformers import BertForSequenceClassification, BertConfig

In [41]:
config=BertConfig.from_pretrained('bert-base-uncased')
config.num_labels=6 #6 options in the coarse-grained labels
model=BertForSequenceClassification(config).to(device)

In [42]:
optim = torch.optim.AdamW(model.parameters(), lr=5e-5)

In [43]:
print(model.named_parameters)

<bound method Module.named_parameters of BertForSequenceClassification(
  (bert): BertModel(
    (embeddings): BertEmbeddings(
      (word_embeddings): Embedding(30522, 768, padding_idx=0)
      (position_embeddings): Embedding(512, 768)
      (token_type_embeddings): Embedding(2, 768)
      (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (encoder): BertEncoder(
      (layer): ModuleList(
        (0): BertLayer(
          (attention): BertAttention(
            (self): BertSelfAttention(
              (query): Linear(in_features=768, out_features=768, bias=True)
              (key): Linear(in_features=768, out_features=768, bias=True)
              (value): Linear(in_features=768, out_features=768, bias=True)
              (dropout): Dropout(p=0.1, inplace=False)
            )
            (output): BertSelfOutput(
              (dense): Linear(in_features=768, out_features=768, bias=True)
              (LayerNo

In [44]:
for name, param in model.named_parameters():
    if 'classifier' not in name: # fix all layers except classifier layer
        param.requires_grad = False
    else:
        print('not me')

not me
not me


In [46]:
import numpy as np

# initialize array to be used
labels = np.zeros(
    (len(trec), max(trec['coarse_label'])+1)
)
# one-hot encode
labels[np.arange(len(trec)), trec['coarse_label']] = 1
labels[:5]

array([[0., 0., 1., 0., 0., 0.],
       [0., 1., 0., 0., 0., 0.],
       [0., 0., 1., 0., 0., 0.],
       [0., 1., 0., 0., 0., 0.],
       [1., 0., 0., 0., 0., 0.]])

In [47]:
labels = torch.Tensor(labels)

In [48]:
class TrecDataset(torch.utils.data.Dataset):
    def __init__(self, tokens, labels):
        self.tokens = tokens
        self.labels = labels

    def __getitem__(self, idx):
        input_ids = self.tokens[idx].ids
        attention_mask = self.tokens[idx].attention_mask
        labels = self.labels[idx]
        return {
            'input_ids': torch.tensor(input_ids),
            'attention_mask': torch.tensor(attention_mask),
            'labels': torch.tensor(labels)
        }

    def __len__(self):
        return len(self.labels)

dataset = TrecDataset(tokens, labels)

In [49]:
loader = torch.utils.data.DataLoader(
    dataset, batch_size=4
)

In [50]:
model.train() #activate training mode

BertForSequenceClassification(
  (bert): BertModel(
    (embeddings): BertEmbeddings(
      (word_embeddings): Embedding(30522, 768, padding_idx=0)
      (position_embeddings): Embedding(512, 768)
      (token_type_embeddings): Embedding(2, 768)
      (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (encoder): BertEncoder(
      (layer): ModuleList(
        (0): BertLayer(
          (attention): BertAttention(
            (self): BertSelfAttention(
              (query): Linear(in_features=768, out_features=768, bias=True)
              (key): Linear(in_features=768, out_features=768, bias=True)
              (value): Linear(in_features=768, out_features=768, bias=True)
              (dropout): Dropout(p=0.1, inplace=False)
            )
            (output): BertSelfOutput(
              (dense): Linear(in_features=768, out_features=768, bias=True)
              (LayerNorm): LayerNorm((768,), eps=1e-12, element

In [51]:
for batch in loader:
    #move to device
    batch_mps = {
        'input_ids': batch['input_ids'].to(device),
        'attention_mask': batch['attention_mask'].to(device),
        'labels': batch['labels'].to(device)
    }
    # initialize calculated gradients (from prev step)
    optim.zero_grad()
    # train model on batch and return outputs (incl. loss)
    outputs = model(**batch_mps)
    # extract loss
    loss = outputs[0]
    # calculate loss for every parameter that needs grad update
    loss.backward()
    # update parameters
    optim.step()

/var/folders/ht/vj5_hz494059tyzm_9_1x92c0000gn/T/ipykernel_2839/3304426640.py:13: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  'labels': torch.tensor(labels)
